In [1]:
import numpy as np
import pandas as pd
import pytest
from tensorflow import Graph, Session

from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
from carla.recourse_methods.catalog.actionable_recourse import ActionableRecourse
from carla.recourse_methods.catalog.cchvae import CCHVAE
from carla.recourse_methods.catalog.cem import CEM
from carla.recourse_methods.catalog.clue import Clue
from carla.recourse_methods.catalog.crud import CRUD
from carla.recourse_methods.catalog.dice import Dice
from carla.recourse_methods.catalog.face import Face
from carla.recourse_methods.catalog.feature_tweak import FeatureTweak
from carla.recourse_methods.catalog.focus import FOCUS
from carla.recourse_methods.catalog.growing_spheres.model import GrowingSpheres
from carla.recourse_methods.catalog.revise import Revise
from carla.recourse_methods.catalog.wachter import Wachter

testmodel = ["ann", "linear"]



/home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mmansour/miniconda3/envs/xai/lib/python3

/home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mmansour/miniconda3/envs/xai/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mmansour/miniconda3

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [8]:
data_name = "adult"
model_type = 'ann'
data = OnlineCatalog(data_name=data_name)

hyperparams_cem = {
    "data_name": data_name,
}

graph = Graph()
with graph.as_default():
    ann_sess = Session()
    with ann_sess.as_default():
        
        model_ann = MLModelCatalog(
                    data=data,
                    model_type=model_type,
                    backend="tensorflow",
                )

        factuals = predict_negative_instances(model_ann, data.df)
        test_factuals = factuals.iloc[:5]

        recourse = CEM(
            sess=ann_sess,
            mlmodel=model_ann,
            hyperparams=hyperparams_cem,
        )

        counterfactuals_df = recourse.get_counterfactuals(factuals=test_factuals)

Train on 34182 samples, validate on 14650 samples
Epoch 1/5
34182/34182 [==============================] - 0s 13us/step - loss: 0.0726 - val_loss: 0.0355
Epoch 2/5
34182/34182 [==============================] - 0s 10us/step - loss: 0.0226 - val_loss: 0.0170
Epoch 3/5
34182/34182 [==============================] - 0s 10us/step - loss: 0.0144 - val_loss: 0.0138
Epoch 4/5
34182/34182 [==============================] - 0s 10us/step - loss: 0.0110 - val_loss: 0.0121
Epoch 5/5
34182/34182 [==============================] - 0s 10us/step - loss: 0.0095 - val_loss: 0.0105


In [9]:
counterfactuals_df

,age,fnlwgt,education-num,capital-gain,capital-loss,...,occupation_Other,relationship_Non-Husband,race_White,sex_Male,native-country_US
0,0.301370,0.044131,0.800000,0.059448,0.0,...,0.0,1.0,1.0,1.0,1.0
1,0.452055,0.048052,0.800000,0.041279,0.0,...,0.0,0.0,1.0,1.0,1.0
2,0.287671,0.137581,0.533333,0.108744,0.0,...,1.0,1.0,1.0,1.0,1.0
3,0.493151,0.150486,0.400000,0.058720,0.0,...,1.0,0.0,0.0,1.0,1.0
4,0.150685,0.220635,0.800000,0.010526,0.0,...,0.0,1.0,0.0,0.0,0.0
